# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [22]:
import os
from glob import glob

# Loading the data into pandas dataframe format
ft_dir = os.getenv("PRICE_DATA")
ft_glob = glob(os.path.join(ft_dir, '**/*.parquet'), recursive = True)
ft_glob = [f for f in ft_glob if os.path.isfile(f)]
df = dd.read_parquet(ft_glob)
df.head()


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year
137315,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,ACN,2001
137316,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,ACN,2001
137317,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,ACN,2001
137318,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,ACN,2001
137319,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,ACN,2001


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [23]:
# Write your code below.

df['Date'] = dd.to_datetime(df['Date'])
df['Close_lag'] = df['Close'].shift(1)
df['Adj Close_lag'] = df['Adj Close'].shift(1)
df['returns'] = (df['Adj Close'] / df['Adj Close_lag']) - 1
df['hi_lo_range'] = df['High'] - df['Low']

dd_feat = df

dd_feat.head()



,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag,Adj Close_lag,returns,hi_lo_range
137315,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,ACN,2001,NaN,NaN,NaN,0.29
137316,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,ACN,2001,15.17,11.404394,-0.010547,0.25
137317,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,ACN,2001,15.01,11.284108,-0.000666,0.46
137318,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,ACN,2001,15.00,11.276587,-0.009333,0.27
137319,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,ACN,2001,14.86,11.171341,0.006056,0.30


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

# Convert to pandas
df_feat = dd_feat.compute()

# Calculate 10-day moving average of returns
df_feat["rolling_avg_return_10d"] = df_feat["returns"].rolling(10).mean()

df_feat.head()



,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag,Adj Close_lag,returns,hi_lo_range,rolling_avg_return_10d
137315,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,ACN,2001,NaN,NaN,NaN,0.29,NaN
137316,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,ACN,2001,15.17,11.404394,-0.010547,0.25,NaN
137317,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,ACN,2001,15.01,11.284108,-0.000666,0.46,NaN
137318,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,ACN,2001,15.00,11.276587,-0.009333,0.27,NaN
137319,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,ACN,2001,14.86,11.171341,0.006056,0.30,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
Dask supports rolling window operations, including .rolling().mean(), provided the data is properly partitioned and indexed.

+ Would it have been better to do it in Dask? Why?
Yes, it would generally be better to compute the moving average in Dask, especially for large datasets, because:
1. Dask operates out-of-core, avoiding memory constraints, 2. It allows parallel computation, 3. Converting to pandas forces all data into memory, which may not scale well. 

Converting to pandas can be acceptable for smaller datasets or exploratory analysis, where simplicity and speed of development matter more than scalability

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.